Used AdaBoost Regression because previously dont know how to approach for Classification Models.
Think this was not in our original plan - JL

# AdaBoost Regression

In [10]:
import re
import io

#General libraries needed
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Libraries for data pre-processing
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn import preprocessing

#For AdaBoost implementation
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

In [11]:
# import csv
hdb_model_data = pd.read_csv('hdb_model_data.csv')
hdb_model_data.sample(5)

,storey_range,floor_area_sqm,no.of bto,resale application,remaining_lease_months,Distance to nearest MRT,Distance to CBD,isMatureEstate,cpi_adjusted_price_per_sqm
74511,0.0000,0.343750,1.000000,1.0,0.141553,0.116269,0.187819,0.0,0.241493
48813,0.0000,0.640625,0.505255,0.0,0.547945,0.179166,0.813232,0.0,0.128851
4071,0.1875,0.182292,0.000000,0.0,0.707763,0.093631,0.410908,1.0,0.409740
27221,0.0625,0.083333,0.505255,0.0,0.299848,0.264530,0.062652,1.0,0.224178
51679,0.0625,0.093750,1.000000,1.0,0.365297,0.208004,0.412458,1.0,0.221127


In [12]:
hdb_model_data.dtypes

storey_range                  float64
floor_area_sqm                float64
no.of bto                     float64
resale application            float64
remaining_lease_months        float64
Distance to nearest MRT       float64
Distance to CBD               float64
isMatureEstate                float64
cpi_adjusted_price_per_sqm    float64
dtype: object

In [13]:
hdb_model_data.sample(5)

,storey_range,floor_area_sqm,no.of bto,resale application,remaining_lease_months,Distance to nearest MRT,Distance to CBD,isMatureEstate,cpi_adjusted_price_per_sqm
53518,0.0625,0.208333,1.0,1.0,0.345510,0.178993,0.531806,1.0,0.233845
21670,0.1250,0.182292,0.0,0.0,0.920852,0.044787,0.629594,1.0,0.367085
7492,0.1250,0.484375,0.0,0.0,0.523592,0.316761,0.660945,1.0,0.175317
69888,0.1875,0.171875,1.0,1.0,0.357686,0.137658,0.301754,1.0,0.303389
8119,0.1875,0.531250,0.0,0.0,0.532725,0.104311,0.600808,1.0,0.164138


In [14]:
X = hdb_model_data.iloc[:,0:8]
y= hdb_model_data.iloc[:,-1]  
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


Used for loop to find best learning rate from 0.1 to 2.0

In [15]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 8,  #no. of trees to be used in the model
                        max_depth = 15,  # maximum number of splits each tree can take. Too low the model will be trained less and have high bias
                        min_samples_split = 2, #every node have 2 subnodes
                        max_features = 6)#maximum features in each tree

In [16]:
# define the model with default hyperparameters
model = AdaBoostRegressor(base_estimator = regressor)

# define the grid of values to search
grid = dict()
grid['n_estimators'] = [50, 100, 150, 200, 250, 300]
grid['learning_rate'] = [0.1, 0.5, 1.0, 1.5, 2.0]

# define the grid search procedure
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=2)

# execute the grid search
grid_result = grid_search.fit(X_train, y_train)

# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

#Use the trained model to predict the test data
y_pred = grid_result.predict(X_test)

Best: 0.938232 using {'learning_rate': 0.1, 'n_estimators': 300}


In [19]:
print("r2 score: " + str(metrics.r2_score(y_test, y_pred)))
print("mean squared error: " + str(metrics.mean_squared_error(y_test, y_pred)))
print("root mean squared error: " + str(metrics.mean_squared_error(y_test, y_pred, squared=False)))

r2 score: 0.9470201392245653
mean squared error: 0.0006093415695594379
root mean squared error: 0.024684844936912972
